In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import os

print("Contents of /kaggle/input:")
for item in os.listdir('/kaggle/input'):
    print(item)

Contents of /kaggle/input:
toronto-emotional-speech-set-tess


In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import librosa

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

def load_data(path):
    data = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(root, file)
                try:
                    audio, sr = librosa.load(file_path, duration=3, sr=22050)
                    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
                    if mfccs.shape[1] < 130:
                        mfccs = np.pad(mfccs, ((0, 0), (0, 130 - mfccs.shape[1])), mode='constant')
                    else:
                        mfccs = mfccs[:, :130]
                    emotion = file.split('_')[-1].split('.')[0]
                    data.append([mfccs, emotion])
                except Exception as e:
                    print(f"Error processing {file}: {str(e)}")
    return pd.DataFrame(data, columns=['features', 'emotion'])

# Load TESS dataset
tess_path = '/kaggle/input/toronto-emotional-speech-set-tess'  # Update this path
print(f"Looking for files in: {tess_path}")
df = load_data(tess_path)

# Prepare features and labels
X = np.array(df['features'].tolist())
y = df['emotion'].values

# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)

# Reshape X for CNN input
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(40, 130, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

# Save the model
model.save('emotion_model_cnn.h5')

# Save the label encoder
import joblib
joblib.dump(le, 'label_encoder_cnn.joblib')

2024-08-11 13:41:01.237290: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 13:41:01.237453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 13:41:01.423414: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Looking for files in: /kaggle/input/toronto-emotional-speech-set-tess


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 19s 121ms/step - accuracy: 0.7458 - loss: 1.3149 - val_accuracy: 0.9955 - val_loss: 0.0090
Epoch 2/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 16s 115ms/step - accuracy: 0.9653 - loss: 0.1128 - val_accuracy: 0.9929 - val_loss: 0.0134
Epoch 3/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 17s 119ms/step - accuracy: 0.9695 - loss: 0.0753 - val_accuracy: 0.9982 - val_loss: 0.0037
Epoch 4/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 16s 114ms/step - accuracy: 0.9796 - loss: 0.0496 - val_accuracy: 1.0000 - val_loss: 3.0046e-04
Epoch 5/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 16s 117ms/step - accuracy: 0.9830 - loss: 0.0454 - val_accuracy: 0.9973 - val_loss: 0.0039
Epoch 6/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 113ms/step - accuracy: 0.9816 - loss: 0.0495 - val_accuracy: 0.9982 - val_loss: 0.0123
Epoch 7/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 17s 118ms/step - accuracy: 0.9838 - loss: 0.0510 - val_accuracy: 0.9982 - val_loss: 0.0058
Epoch 8/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 16s 116ms/step - accuracy: 0.9932 - los

['label_encoder_cnn.joblib']